# Environment Setup

In [2]:
# !git clone https://github.com/anantgupta129/TorcHood.git
# %cd TorcHood  
# !git checkout trf_speed_up


In [3]:
%cd era/TorcHood

/home/watcherai/.dev/era/TorcHood


In [4]:
from IPython.display import clear_output

!python setup.py sdist
!pip install .

clear_output()
%cd ..

/home/watcherai/.dev/era


# Importing Libaries

In [5]:
import warnings
warnings.filterwarnings("ignore")

import os 
import torch

import lightning.pytorch as pl
from torchood.configs.bilang_config import get_config
from torchood.data.opus_datamodule import OpusBooksDataModule
from torchood.models.bilang_module import BiLangLitModule

In [6]:
config = get_config()
config.update({
  "project_name" : "Transformers-BiLang", # Wandb project name
  "run_name" : "dp+ps+ocp3phase", # wandb run name
  "batch_size" : 32,
  "lang_src" : "en",
  "lang_tgt" : "fr",
  "seq_len" : 160,
  "num_epochs" : 10,
  "lr": 1e-3
})

In [7]:
datamodule = OpusBooksDataModule(config, 8, True)

In [8]:
datamodule.prepare_data()
datamodule.setup("")

Tokenizer found. Loading from file...
Tokenizer found. Loading from file...
[x] Maximum Length of Sentence:-
	[*] source sentence: 471
	[*] target sentence: 482


In [9]:
model = BiLangLitModule(config["lr"], config, datamodule.tokenizer_src, datamodule.tokenizer_tgt, parameter_sharing=True)

In [10]:
from lightning.pytorch.callbacks import (
    LearningRateMonitor,
    RichProgressBar,
)
from lightning.pytorch.loggers import WandbLogger


callbacks = [LearningRateMonitor(logging_interval="step")]
logger = WandbLogger(project=config["project_name"], name=config["run_name"])
trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    max_epochs=config["num_epochs"],
    accelerator = "gpu",
)
trainer.fit(model=model, datamodule=datamodule)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anantgupta129. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type             | Params
-----------------------------------------------------
0 | net             | Transformer      | 60.8 M
1 | criterion       | CrossEntropyLoss | 0     
2 | char_error_rate | CharErrorRate    | 0     
3 | word_error_rate | WordErrorRate    | 0     
4 | bleu_score      | BLEUScore        | 0     
--------------------------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
